# Prepared Sequence Data

In [1]:
from alarms2 import *
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# def findChatteringsv2(alarms, chattering_timedelta_threshold=60.0, chattering_count_threshold=3):
#     """Find the chatterings in an alarms list from the same source. 

#     Parameters
#     ----------
#     alarms  : list of dict
#         A list of alarms from the same source. 
#     chattering_timedelta_threshold : float, optional
#         Duration in seconds for which to finde cattering alarms, by default 60.0 seconds.
#     chattering_count_threshold : int, optional
#         Threshold for minimum number of alarms to be activated in duration of chattering_timedelta_threshold, by default 3

#     Returns
#     ----------
#     chattering : dict
#         It contains the StartTime as key of chattering, and a dict as a value which is 
#         consits of an index and a count of of alarms chattered within chattering_timedelta_threshold next 
#         to it.  
#     """
#     alarms_without_chatters = []
#     chattering = {}
#     alarms = [alarm for alarm in sorted(alarms, key=lambda arg: arg["StartTime"], reverse=False)]
#     i = 0
#     j = 0

#     while i < (len(alarms)):
#         prev_start = alarms[i]["StartTime"]
#         prev_end = alarms[i]["EndTime"]
#         count_alarms = 0
#         j = i + 1
#         while j < len(alarms):
#             next_start = alarms[j]["StartTime"]
#             next_end = alarms[j]["EndTime"]

#             # this assert is very important: the prev alarm has to turn off before the start of
#             # the next one
#             if prev_end > next_start:
#                 print(alarms[i]["SourceName"], alarms[j]["SourceName"])
#                 print(prev_start,prev_end, next_start,next_end)
#             assert(prev_start <= next_start)
#             assert(prev_end <= next_start)
#             assert(prev_end <= next_end)

#             delta = timedelta.total_seconds(next_start - prev_start)
#             assert (delta >= 0)
#             if delta > chattering_timedelta_threshold:
#                 break
#             count_alarms += 1
# #             print(time_delta, "count ++ ", count, t_prev, t_next)
#             j += 1

#         alarms_without_chatters.append(alarms[i])
#         if count_alarms >= chattering_count_threshold:
#             chattering[prev_start] = {"index": i, "count": count_alarms}
#             i = j
#         else:
#             i += 1
        

#     return chattering, alarms_without_chatters

In [3]:
path = "./data/new/"

In [4]:
alarms_fname = "formatted-all-month-alarms.csv" 
# operator_fname = "operator-all-month-actions.csv"

df_main_alarms = pd.read_csv(path + alarms_fname, low_memory=False ,parse_dates=["StartTime", "EndTime"])
df_main_alarms["TimeDelta"] = df_main_alarms["EndTime"] - df_main_alarms["StartTime"]
df_main_alarms["TimeDelta"] = df_main_alarms["TimeDelta"].apply(lambda arg: timedelta.total_seconds(arg)) 
df_main_alarms["Month"] = df_main_alarms["StartTime"].apply(lambda arg: arg.month)

# df_main_actions = pd.read_csv(path + operator_fname, low_memory=False ,parse_dates=["EventTime"])
# df_main_actions["Month"] = df_main_actions["EventTime"].apply(lambda arg: arg.month)

## Filtering Some Uncessary Alarms From Orignal Data

In [5]:
filter_short_alarms = [20]
min_alarms_per_source = 50 # include only these alarms
igonre_snames = [] # "47TI1713" ignore these alarms

# df_temp = df_main_alarms
df_f2 = df_main_alarms[df_main_alarms["TimeDelta"]>filter_short_alarms[0]]
len( df_main_alarms["SourceName"].unique()),len(df_f2["SourceName"].unique())

source2count = dict(df_f2["SourceName"].value_counts())
source2count = {k:v for k,v in source2count.items() if v>=min_alarms_per_source}

print(len(source2count))

df_rnn = df_main_alarms[(df_main_alarms["TimeDelta"]>=filter_short_alarms[0]) & (df_main_alarms["SourceName"].isin(source2count.keys())) & (~df_main_alarms["SourceName"].isin(igonre_snames))]

source2count = dict(df_rnn["SourceName"].value_counts())
source2count = {k:v for k,v in source2count.items() if v>=min_alarms_per_source}
print(len(source2count))
print(source2count)

169
169
{'47TI931A': 58368, '47TI1713': 16954, '47PI1734': 10973, '48PI2025': 10563, '47LIC005': 3636, '47FI1575': 3004, '48FIX2040': 2627, '48TIC2027': 2627, '47PIC1755': 2486, '47PDI1746': 2403, '47FI1568': 2281, '48XL001-ANN': 2118, '47FLI-039': 1962, 'SIO-18XA001': 1947, '19LI105A': 1939, '48XL002-ANN': 1851, '47PDI1747': 1658, '47TI003B': 1394, '47LI003': 1317, '47TI003A': 1078, '47LIC1503': 1033, '47TIC3520': 967, '47TI002C': 941, '19G402-ANN': 923, '19XL101-ANN': 923, '47PDI003': 740, '47PIC2033A': 727, '47PI2029C': 717, '47PIC1505': 701, '48TIC034': 698, '47PI2039B': 688, '47PI2039A': 668, '47FLI-042': 652, '47FI043': 647, '47XL015-ANN': 640, '47XL016-ANN': 633, '47PI2039C': 629, '47PI2029B': 604, '47TAL003AB-ANN': 595, '47PI2029A': 595, '48FI2003A': 562, '47TI1769': 552, '47TI1512A': 509, '47PDIC1759': 465, '47TI1512B': 459, '47TI1512ABC': 436, '47TI1723': 428, '47TI1512C': 402, '48LI015': 399, '47FLI-037': 354, '47FI1005': 353, '47PIC2064': 325, '47LIC3103': 320, '47HCI-003':

In [6]:
 df_rnn = df_rnn[df_rnn["SourceName"].isin(source2count.keys())]
#  for sname in df_rnn["SourceName"].unique():
#     print(">>",sname)
#     df_sname = df_rnn[df_rnn["SourceName"]==sname]
#     for condition in df_sname["Condition"].unique():
#         df_condition = df_sname[df_sname["Condition"]==condition]
#         chats, without_chatter = findChatteringsv2(df_condition.to_dict(orient="records"))
#         print(len(chats),len(without_chatter))

### Functions to preprocess the data

 Removing short sequences


In [169]:
from functools import partial
from sklearn.model_selection import train_test_split
import itertools

def getSequenceByMonth(df,month,seq_duration_gap,filter_short_seq,filter_long_seq):
    print(f">> Seq By Month:{month}, duration to next seq: {seq_duration_gap}, ignore seq len: {filter_short_seq}")

    list_of_sequences = []
    max_len = 0
    min_len = 9999   
    df_month = df[df["Month"]==month]
    assert 1 == len(df_month["Month"].unique())
    
    alarms= df_month.to_dict(orient="records")
    alarms = [alarm for alarm in sorted(alarms, key=lambda arg: arg["StartTime"], reverse=False)] # sorting
    # print(alarms[:2])
    i =0
    j= 0
    while i <len(alarms):
        prev_start = alarms[i]["StartTime"]
        seq = []
        seq.append(alarms[i])
        j = i+1
        while j < len(alarms):    
            next_start = alarms[j]["StartTime"]
            delta = timedelta.total_seconds(next_start - prev_start)
            # print(delta)
            assert delta >= 0
            if delta >= seq_duration_gap:
                break
            if len(seq) > filter_long_seq:
                # seq = [alarm for alarm in sorted(seq, key=lambda arg: arg["StartTime"], reverse=False)]
                # seq = [alarm["SourceName"] for alarm in seq]
                # list_of_sequences.append(seq)
                # print("<<",len(seq))
                break

            seq.append(alarms[j])
            j += 1
        i = j
        if len(seq)>max_len:
            max_len = len(seq)
        if len(seq)<min_len:
            min_len = len(seq)
        

        if len(seq)>=filter_short_seq:
            seq = [alarm for alarm in sorted(seq, key=lambda arg: arg["StartTime"], reverse=False)]
            seq = [alarm["SourceName"] for alarm in seq]
            list_of_sequences.append(seq)
    
    # list_of_sequences = [l for l in list_of_sequences if (len(l)>filter_short_alarms)]
    print(f">> Min Seq Len: {min_len}, max seq len: {max_len}")
    return list_of_sequences


#Doing Padding
def padding(seq_length,arr):
    
    if len(arr) < seq_length:
        return ["NoName" for i in range(seq_length-len(arr))] + arr
    elif len(arr) > seq_length: # if bigger than seq_length use earliers
        return arr[0:seq_length]

    return arr

def getAllMonths2Seqs(df,seq_length,seq_time_gap,seq_ignore_len,filter_long_seq):
    print(f">> All Months: Seq_len = {seq_length}")
    month2alarms = {}

    for month in df["Month"].unique():
        li_of_seqs = getSequenceByMonth(df,month,seq_time_gap,seq_ignore_len,filter_long_seq) 
        f = partial(padding,seq_length)
        month2alarms[month] = [f(l) for l in li_of_seqs]
    
    return month2alarms


def getTrainAndValidationData(df,seq_length,seq_time_gap,seq_ignore_len,filter_long_seq,test_size,shuffle=False):
    print(f">> Spliting Data Size (valid%): {test_size}")
    train_data = []
    valid_data = []
    months2seq = getAllMonths2Seqs(df,seq_length=seq_length,seq_time_gap=seq_time_gap,seq_ignore_len=seq_ignore_len,filter_long_seq=filter_long_seq)

    for month in df["Month"].unique():
      train,valid = train_test_split(months2seq[month],test_size=test_size,shuffle=shuffle)
      train_data += train
      valid_data += valid

    return train_data, valid_data  

def encodeData(mydict,l):
    return [mydict[e] for e in l]

def getInputsAndTargets(encoded_alarms):
    input_seqs_train = []
    target_seqs_train = []

    for i in range(len(encoded_alarms)):
        # remove the last char from input seq
        input_seqs_train.append(encoded_alarms[i][:-1])

        # remove the first char from input seq
        target_seqs_train.append(encoded_alarms[i][1:])
        # print(f"orignal={encoded_alarms[i]} \n Input Seq={input_seqs_train[i]}, \n Target Seq = {target_seqs_train[i]}")

    inputs = []
    targets = []
    for l in input_seqs_train:
        inputs = inputs+l

    for l in target_seqs_train:
        targets = targets + l


    # for row in range(len(input_seqs_train)): 
    row = 0

    # print(inputs[row*(seq_length-1):row*(seq_length-1)+ seq_length-1],targets[row*(seq_length-1):row*(seq_length-1)+ seq_length-1])
    print(input_seqs_train[row], target_seqs_train[row])
    return inputs, targets, input_seqs_train, target_seqs_train

def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    # print(arr.size) # total number of memory locations 
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    indexes = np.arange(one_hot.shape[0]), arr.flatten()
    # print(indexes)
    one_hot[indexes] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

def get_batches(arr1,arr2, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    t = arr1.size//(batch_size * seq_length)
    
    n_batches =  t
    
    ## TODO: Keep only enough characters to make full batches
    arr1 =  arr1[:batch_size*(seq_length*n_batches)]
    arr2 =  arr2[:batch_size*(seq_length*n_batches)]

    ## TODO: Reshape into batch_size rows
    arr1 = arr1.reshape((batch_size,-1))
    arr2 = arr2.reshape((batch_size,-1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr1.shape[1], seq_length):
        # The features
        x = arr1[:,n:n+seq_length]
        y = arr2[:,n:n+seq_length]
        
        
        yield x, y
    

# sequences = getSequenceByMonth(df_rnn,3)
# # sequences 

# print(len(sequences))
# print(max([len(seq) for seq in sequences]),min([len(seq) for seq in sequences]))
# # print([len(seq) for seq in sequences])
# print(sequences[:2])

## Mention the Sequence Length => how many alarms you want to put in a sequence
Any change need to rerun from here

In [258]:
seq_length = 5
ingore_short_seq_len = 4
long_len = seq_length 
duration_from_1_seq_to_next = 60*10 # duration in seconds
test_size = 0.2
shuffle = False

print("    ===== Without Ignoring Short Seq==========") 
train_data, valid_data = getTrainAndValidationData(df_rnn,seq_length=seq_length,seq_time_gap=duration_from_1_seq_to_next,seq_ignore_len=0,filter_long_seq=long_len,test_size=test_size,shuffle=shuffle)

print(f">> Train Size: {len(train_data)}, Validation Size: {len(valid_data)}")  


print("      ===== Ignoring Short Seq==========") 
train_data, valid_data = getTrainAndValidationData(df_rnn,seq_length=seq_length,seq_time_gap=duration_from_1_seq_to_next,seq_ignore_len=ingore_short_seq_len, filter_long_seq=long_len,test_size=test_size,shuffle=shuffle)

print(f">> Train Size: {len(train_data)}, Validation Size: {len(valid_data)}")  
print(valid_data[1])

===== Without Ignoring Short Seq==========
>> Spliting Data Size (valid%): 0.2
>> All Months: Seq_len = 5
>> Seq By Month:3, duration to next seq: 600, ignore seq len: 0
>> Min Seq Len: 1, max seq len: 6
>> Seq By Month:4, duration to next seq: 600, ignore seq len: 0
>> Min Seq Len: 1, max seq len: 6
>> Seq By Month:5, duration to next seq: 600, ignore seq len: 0
>> Min Seq Len: 1, max seq len: 6
>> Seq By Month:6, duration to next seq: 600, ignore seq len: 0
>> Min Seq Len: 2, max seq len: 6
>> Train Size: 22256, Validation Size: 5567
      ===== Ignoring Short Seq==========
>> Spliting Data Size (valid%): 0.2
>> All Months: Seq_len = 5
>> Seq By Month:3, duration to next seq: 600, ignore seq len: 4
>> Min Seq Len: 1, max seq len: 6
>> Seq By Month:4, duration to next seq: 600, ignore seq len: 4
>> Min Seq Len: 1, max seq len: 6
>> Seq By Month:5, duration to next seq: 600, ignore seq len: 4
>> Min Seq Len: 1, max seq len: 6
>> Seq By Month:6, duration to next seq: 600, ignore seq len

## Preparing Vocab and conversion from vocab2int and into2vocab

In [259]:

vocab = set(list(itertools.chain.from_iterable(train_data+valid_data)))
int2vocab = dict(enumerate(vocab))
vocab2int = {v:k for k,v in int2vocab.items()}
assert len(vocab2int) == len(vocab)

f = partial(encodeData,vocab2int)
encoded_train_alarms = [f(l) for l in train_data]
encoded_valid_alarms = [f(l) for l in valid_data]


train_features = np.array(encoded_train_alarms)
train_features.shape


(22060, 5)

## Preparing Data To Use mini batches

In [260]:

row = 0 # this row should always be zero because in function it is zero

print(">> ============ Training Set ==============")
train_inputs, train_targets, _, _ = getInputsAndTargets(encoded_train_alarms)
print(train_inputs[row*(seq_length-1):row*(seq_length-1)+ seq_length-1],train_targets[row*(seq_length-1):row*(seq_length-1)+ seq_length-1])

print(">> =========== Validation Set ==============")
valid_inputs, valid_targets, _, _ = getInputsAndTargets(encoded_valid_alarms)    
print(valid_inputs[row*(seq_length-1):row*(seq_length-1)+ seq_length-1],valid_targets[row*(seq_length-1):row*(seq_length-1)+ seq_length-1])

print(">> Ignore Seq", vocab2int["NoName"])

>> ============ Training Set ==============
[16, 16, 16, 152] [16, 16, 152, 89]
[16, 16, 16, 152] [16, 16, 152, 89]
>> =========== Validation Set ==============
[16, 141, 113, 144] [141, 113, 144, 113]
[16, 141, 113, 144] [141, 113, 144, 113]
>> Ignore Seq 147


## Testing Mini Batches

Below output should be similar to above 1

In [261]:
batches = get_batches(np.array(train_inputs),np.array(train_targets),batch_size=2, seq_length=seq_length-1)
x, y = next(batches)

print(x.shape, y.shape)

# printing out the first 10 items in a sequence
print('x\n', x)
print('\ny\n', y)



(2, 4) (2, 4)
x
 [[ 16  16  16 152]
 [ 42  16  16 137]]

y
 [[ 16  16 152  89]
 [ 16  16 137  16]]


## RNN Network

In [262]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [263]:
class AlarmRNN(nn.Module):
    
    def __init__(self,tokens,int2vocab,vocab2int, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super(AlarmRNN,self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens # vocab
        # int2vocab = dict(enumerate(self.chars))
        # vocab2int = {v:k for k,v in int2vocab.items()}

        self.int2char = int2vocab #dict(enumerate(self.chars))
        self.char2int = vocab2int  #{ch: ii for ii, ch in self.int2char.items()}
                
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=self.n_hidden, num_layers=self.n_layers,dropout=self.drop_prob, batch_first=True)
        self.droput = nn.Dropout(p=self.drop_prob)
        self.fc = nn.Linear(in_features=self.n_hidden, out_features=len(self.chars))   
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        out, hidden = self.lstm(x,hidden)
        out = self.droput(out)
        # Contiguous variables: If you are stacking up multiple LSTM outputs, it may be necessary to use .contiguous() to reshape the output.
        out = out.contiguous().view(-1,self.n_hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [264]:
def train(net,data,seq_length,batch_size,epochs=10, lr=0.001, clip=5, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    print(f"Batch Size ={batch_size}, seq_length={seq_length}")

    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    # val_idx = int(len(data)*(1-val_frac))
    # data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    loss = 0
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data["train_inputs"], data["train_targets"], batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
        # loss stats with validation 
        if (e+1) % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for x, y in get_batches(data["valid_inputs"],data["valid_targets"],batch_size, seq_length):
                # One-hot encode our data and make them Torch tensors
                x = one_hot_encode(x, n_chars)
                x, y = torch.from_numpy(x), torch.from_numpy(y)
                
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])
                
                inputs, targets = x, y
                if(train_on_gpu):
                    inputs, targets = inputs.cuda(), targets.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output, targets.view(batch_size*seq_length).long())
            
                val_losses.append(val_loss.item())
            
            net.train() # reset to train mode after iterationg through validation data
            
            print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.4f}...".format(loss.item()),
                    "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [265]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        if(train_on_gpu):
            net.cuda()
        else:
            net.cpu()

        net.eval() # I have written this

        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

def sample(net,h,input_sequence,prime,top_k=None):
    # print(f">>Prime:{prime}, Input Seq: {input_sequence}")

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    char = None
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    output = None
    for ch in input_sequence:
        # print(">> ", ch)
        # if ch == "NoName":
        #     continue
        output, h = predict(net, ch, h, top_k=top_k)
    
    
    return output,h


In [266]:
def getAccuracy(inputs,targets,net):
    correct = 0
    wrong = 0 
    hidden = net.init_hidden(1)
    prime = ['19LI105A', '47TI931A', '48XL001-ANN', '19LI105A', '47TI931A']
    
    for i in range(len(inputs)):
        # output,hidden =sample(net,hidden,input_sequence=inputs[i], prime=prime, top_k=None)
        for j in  range(len(inputs[0])):
            output,hidden =predict(net,char= inputs[i][j], h=hidden,top_k=None)
            
            target = targets[i][j]
            # print(">> output", output)
            # print(">> Target", target)
            if output == target and target != "NoName":
                correct += 1
            elif output != target and target != "NoName":
                wrong += 1

    accuracy = correct/(wrong+correct)
    return accuracy*100

In [267]:
_,_,myinputs_t, mytargets_t = getInputsAndTargets(train_data)
train_aacuracy_fun = partial(getAccuracy, myinputs_t, mytargets_t) 

_,_,myinputs_v, mytargets_v = getInputsAndTargets(valid_data)
validation_aacuracy_fun = partial(getAccuracy, myinputs_v, mytargets_v) 


['47TI931A', '47TI931A', '47TI931A', '48FIX2040'] ['47TI931A', '47TI931A', '48FIX2040', '48TIC2027']
['47TI931A', '47PDI1747', '47PI1734', '47LIC3103'] ['47PDI1747', '47PI1734', '47LIC3103', '47PI1734']


In [268]:
## TODO: set your model hyperparameters
# define and print the net
torch.cuda.empty_cache()

n_hidden=256
n_layers=2
n_epochs = 200 # start small if you are just testing initial behavior
batch_size = 1024
data = {"train_inputs":np.array(train_inputs), "train_targets": np.array(train_targets), "valid_inputs":np.array(valid_inputs), "valid_targets":np.array(valid_targets)}

net = AlarmRNN(vocab,int2vocab,vocab2int,n_hidden, n_layers,drop_prob=0.0)
print(net)

# train the model
train(net,data, seq_length=seq_length-1, batch_size=batch_size, epochs=n_epochs,lr=0.002, print_every=10)

train_accuracy = 0
valid_accuracy = 0
# train_accuracy = train_aacuracy_fun(net)
# valid_accuracy = validation_aacuracy_fun(net)
print(f"Accuracy: Train =  {train_accuracy}, valid = {valid_accuracy}, 0 means intentally is set to zero")

AlarmRNN(
  (lstm): LSTM(170, 256, num_layers=2, batch_first=True)
  (droput): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=256, out_features=170, bias=True)
)
Batch Size =1024, seq_length=4
Epoch: 10/200... Step: 210... Loss: 2.9479... Val Loss: 2.9784
Epoch: 20/200... Step: 420... Loss: 2.5433... Val Loss: 2.5605
Epoch: 30/200... Step: 630... Loss: 2.2749... Val Loss: 2.3960
Epoch: 40/200... Step: 840... Loss: 2.1275... Val Loss: 2.2727
Epoch: 50/200... Step: 1050... Loss: 2.0456... Val Loss: 2.2872
Epoch: 60/200... Step: 1260... Loss: 1.9758... Val Loss: 2.3073
Epoch: 70/200... Step: 1470... Loss: 1.8270... Val Loss: 2.3549
Epoch: 80/200... Step: 1680... Loss: 1.7371... Val Loss: 2.4334
Epoch: 90/200... Step: 1890... Loss: 1.5995... Val Loss: 2.5289
Epoch: 100/200... Step: 2100... Loss: 1.4340... Val Loss: 2.6431
Epoch: 110/200... Step: 2310... Loss: 1.3365... Val Loss: 2.8233
Epoch: 120/200... Step: 2520... Loss: 1.1814... Val Loss: 2.8749
Epoch: 130/200... Step: 2730..

In [269]:
train_accuracy = train_aacuracy_fun(net)
print(f"Accuracy: Train =  {train_accuracy}, valid = {valid_accuracy}, 0 means intentally is set to zero")
# valid_accuracy = validation_aacuracy_fun(net)
print(f"Accuracy: Train =  {train_accuracy}, valid = {valid_accuracy}, 0 means intentally is set to zero")

Accuracy: Train =  65.74682683590208, valid = 0, 0 means intentally is set to zero
Accuracy: Train =  65.74682683590208, valid = 0, 0 means intentally is set to zero


In [270]:
hidden = net.init_hidden(1)
prime= ['NoName', 'NoName', 'NoName']
input_seq = ['19LI105A', '47TI931A', '48XL001-ANN', '19LI105A', '47TI931A']
target = '47LIC3408'
output =sample(net,hidden,input_sequence=input_seq, prime=prime, top_k=None)
print(f">> Target={target}, Output = {output}")

>> Target=47LIC3408, Output = ('47TI1713', (tensor([[[ 2.8783e-01,  1.6483e-01, -3.4231e-01,  7.4585e-02,  1.6151e-01,
           4.0563e-03, -3.2046e-02,  1.4720e-02, -4.5031e-01, -5.3719e-01,
          -9.9898e-02,  1.2104e-01,  9.2255e-01, -4.1955e-01, -1.0587e-01,
           1.5443e-01,  1.6324e-01, -3.7993e-02,  3.1831e-02, -1.2007e-02,
          -4.5332e-01,  7.7906e-01,  3.2840e-01, -4.2610e-01,  3.5148e-01,
           6.2308e-01,  4.0928e-01,  2.0851e-03,  7.4351e-01,  5.6429e-01,
           8.3267e-01, -4.4880e-01, -1.5362e-01, -3.1143e-01,  6.6036e-02,
          -5.0778e-02, -1.9771e-01, -7.3669e-01,  5.4859e-02, -2.0804e-01,
          -4.5646e-02,  1.8821e-01, -1.8868e-02,  2.7489e-02, -9.2930e-02,
          -8.5812e-02, -2.1644e-02, -5.1700e-01, -2.1295e-01, -6.6207e-01,
          -6.6450e-02, -1.7746e-02, -2.1170e-02, -1.8510e-01,  1.0439e-01,
          -2.4383e-01,  3.9228e-01, -2.8720e-01,  4.0876e-03, -8.7549e-01,
          -8.6007e-01,  1.9680e-02, -1.3628e-03, -1.9164

In [271]:
# import torch.nn as nn

# class AlarmRNN(nn.Module):
    
#     def __init__(self,vocab_size, output_size,embedding_dim=80, hidden_dim=256, n_layers=2, drop_prob=0.5, batch_size=50):
#         super(AlarmRNN,self).__init__()

#         self.output_size = output_size
#         self.drop_prob = drop_prob
#         self.n_layers = n_layers

#         self.hidden_dim = hidden_dim
#         self.batch_size = batch_size

#         # self.lr = lr
        
#         # creating character dictionaries
#         # self.chars = tokens # vocab
#         # self.int2char = dict(enumerate(self.chars))
#         # self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
#         ## TODO: define the layers of the model
        
#         self.embedding = nn.Embedding(vocab_size,embedding_dim)
#         self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_dim, num_layers=self.n_layers,dropout=self.drop_prob, batch_first=True)
        
#         self.droput = nn.Dropout(p=self.drop_prob)
#         self.fc = nn.Linear(in_features=self.hidden_dim, out_features=output_size) 
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, x, hidden):
#         ''' Forward pass through the network. 
#             These inputs are x, and the hidden/cell state `hidden`. '''
                
#         ## TODO: Get the outputs and the new hidden state from the lstm
#         x = x.long()
#         embeds = self.embedding(x)
#         out, hidden = self.lstm(embeds,hidden)
#         out = self.droput(out)
#         # Contiguous variables: If you are stacking up multiple LSTM outputs, it may be necessary to use .contiguous() to reshape the output.
#         out = out.contiguous().view(-1,self.hidden_dim)
#         print(">> out shape", out.size())
#         out = self.fc(out)
#         print(">> out shape before softmax", out.size())
#         out = self.softmax(out)
#         print(">> out shape after softmax", out.size())
#         # return the final output and the hidden state
#         return out, hidden
    
    
#     def init_hidden(self):
#         ''' Initializes hidden state '''
#         # Create two new tensors with sizes n_layers x batch_size x n_hidden,
#         # initialized to zero, for hidden state and cell state of LSTM
        
#         weight = next(self.parameters()).data
        
#         if (train_on_gpu):
#             hidden = (weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_().cuda(),
#                   weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_().cuda())
#         else:
#             hidden = (weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_(),
#                       weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_())
        
#         return hidden
        